pypackmol examples
----
The pypackmol module is contained inside pypackmol.py in the nanoreactor directory

In [1]:
import pypackmol

The **Packmol** class represent a packmol run.  Input files are tempfiles hidden from the user so for all in tent and purposes this behaves like a normal python component.  Argument `dimension` specifies the radius of the sphere.  You can also add any `packmol` options here in the argument.

By default the `packmol` command line is just

    $ packmol

So you will have to put configure your PATH variable.  Alternatively you can use `command_line` argument to specify the command line.  

Here we use SMILES strings to add 10 water, 10 methane and 10 ammonia molecules to the system.

In [2]:
pm=pypackmol.Packmol(dimension=8)
pm.add_structure('O',count=10)
pm.add_structure('C',count=10)
pm.add_structure('N',count=10)

The `pack()` function calls packmol.  By default the packed geometry is saved in `"packed.xyz"`, but you can specify other path through the `output` argument.  The return is a `dict` with two fields.  *"filename"* contains the name of the packed geometry file, and *"packmol output"* contains the packmol printout.

In [3]:
result=pm.pack(output="CNOsystem.xyz")

The default distance threshold is `tolerance=3.0` so the molecules are fairly separated.  SMILES are optimized to 3D structures with *Universal Force Field*.  *Merck* force fields are probably better but it cannot treat atoms like `Cs` so it is not set as default.

In [4]:
from imolecule import draw
draw(result['filename'])

If you change your mind you can use `set_options()` method to change options.  Here we specify the Merck force fields to be used to construct 3D geometries from SMILES, and changed `packmol` parameters a bit.

In [5]:
pm.set_options(opt_force_field='mmff94',tolerance=3.5,dimension=5,nloop=50)

Now we try packing again with these new parameters.  `pack()` will throw an error when packmol fails.  Here the radius is too small and packing is not possible.  

**TODO** We should in the future implement **Laszlo**'s automatic radius setting as a method.  We can also start with the heuristic radius then do trial and error until packing is successful.  

In [6]:
result2=pm.pack(output="smaller.xyz")

FailToPack: Packing failed.  Cannot fulfill tolerance constraints. Best result found in [smaller.xyz]

The `autosize()` method can be used to automatically determine the minimum radius of the packing area.  It simply perform a series of trial `packmol` runs with increasing radius until packing is successful.  This may take a minute for larger systems.

In [7]:
pm.clear()
pm.set_options(output="furfural.xyz",nloops=30,opt_force_field='uff')
pm.add_structure("o1cccc1C(=O)[O-]",2)
pm.add_structure("oc([O-])[O-]",6)
pm.add_structure("O=C=O",5)
pm.add_structure("[Cs+]",12)
radius=pm.autosize(min_radius=5,increment=0.1)

Packing successful with radius 6.0


In [8]:
draw(pm.last_result()['filename'])

There are some limited documentations and they can be accessed through the `help` command if you are in **IPython** or interactive shell.  

In [9]:
help(pypackmol)

Help on module pypackmol:

NAME
    pypackmol

FILE
    /home/stanford/toddmtz/users/zhuxl/nanoreactor-dev/pypackmol/src/pypackmol.py

DESCRIPTION
    PyPackmol is a Python wrapper for packmol program that provide simple packing of molecules. 
    Note that packmol needs to be installed separately.  
    Molecules can be given in terms of input files (xyz, etc), or SMILES strings.  
    Open Babel is required to do the conversion if the input format is not xyz.

CLASSES
    __builtin__.object
        Packmol
    exceptions.Exception(exceptions.BaseException)
        FailToPack
    
    class FailToPack(exceptions.Exception)
     |  #Exception class for packing failure
     |  
     |  Method resolution order:
     |      FailToPack
     |      exceptions.Exception
     |      exceptions.BaseException
     |      __builtin__.object
     |  
     |  Data descriptors defined here:
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  